In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers,Sequential
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
train = r""
val = r""
img_size = (224,224)

In [ ]:
train_d = tf.keras.utils.image_dataset_from_directory(train,image_size = img_size)
val_d = tf.keras.utils.image_dataset_from_directory(val,image_size = img_size)
names = train_d.class_names

In [ ]:
train_d = train_d.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)
val_d = train_d.cache().prefetch(tf.data.AUTOTUNE)

In [ ]:
base = tf.keras.applications.EfficientNetV2B0(include_top = False,weights = "imagenet",input_shape = (224,224,3))
base.trainable = False

In [ ]:
model = Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(len(names),activation = 'softmax')
])

In [ ]:
model.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
history = model.fit(train_d,validation_data = val_d,epochs = 5)

In [ ]:
print("val Acc : ",model.evaluate(val_d)[1]*100,"%")

In [ ]:
for imgs,lbls in val_d.take(1):
    preds = tf.argmax(model.predict(imgs),axis = 1)
    plt.figure(figsize = (8,8))
    for i in range(9):
        plt.subplot(3,3,i+1)
        plt.imshow(imgs[i].numpy().astype("uint8"))
        plt.title(f"p : {names[preds[i]]}\nT: {names[lbls[i]]}")
        plt.axis('off')
    plt.show()

In [ ]:
eval_ds = tf.keras.utils.image_dataset_from_directory(val, image_size=img_size, shuffle=False)

y_true = tf.concat([y for _, y in eval_ds], axis=0).numpy()
y_pred = tf.argmax(model.predict(eval_ds), axis=1).numpy()

print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=names))

cm = confusion_matrix(y_true, y_pred)
ConfusionMatrixDisplay(cm, display_labels=names).plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix - Cats vs Dogs")
plt.show()